# Open Data Example

In this example, we consider [CMS open dataset](https://energyflow.network/docs/datasets/) for jets. At the particle level, we consider the sum of the leading and subleading jets as our observable. At the detector level, the observables include both the sum and the difference of the leading and subleading jets.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import glob
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import seaborn as sns

# Energy-flow package for CMS Open Data loader
import energyflow as ef
from energyflow.utils import remap_pids

# Pytorch
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split, DataLoader
from torch import nn, optim

# POF functions
import utils
import profile_omnifold as pof

dvc = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {dvc} device")

## Load Data

In [6]:
# specify the nuisance parameter
theta = 1.7

In [7]:
def load_data(cache_dir, pt_lower, pt_upper, eta, quality, pad, x_dim = 3, momentum_scale = 250, n = 100000, amount = 1, max_particle_select = None, frac = 1.0):

    # Load data
    specs = [f'{pt_lower} <= gen_jet_pts <= {pt_upper}', f'abs_gen_jet_eta < {eta}', f'quality >= {quality}']
    # specs = [f'{pt_lower} <= jet_pts <= {pt_upper}', f'abs_jet_eta < {eta}', f'quality >= {quality}']
    sim = ef.mod.load(*specs, cache_dir = cache_dir, dataset='sim', amount= amount, store_gens = False, subdatasets=['SIM600_Jet300_pT375-infGeV'])

    # Gen_pt for Y
    Y1 = sim.jets_f[:,sim.gen_jet_pt]
    Y = np.zeros((Y1.shape[0], 1), dtype = np.float32 )
    Y[:,0] = Y1 / momentum_scale

    # Sim_pt for X
    X = np.zeros((Y1.shape[0],3), dtype = np.float32)
    event_ids = np.zeros((Y1.shape[0],1), dtype = np.int32)
    X[:,0] = sim.jets_f[:,sim.jet_pt] / momentum_scale
    X[:,1] = sim.jets_f[:,sim.jet_eta]
    X[:,2] = sim.jets_f[:,sim.jet_phi]
    event_ids = sim.jets_i[:,sim.evn]

    Y = Y[:n]
    X = X[:n]
    event_ids = event_ids[:n]

    # Trim and shuffle
    if max_particle_select is not None:
        dataset = dataset[particle_counts < max_particle_select]
        Y = Y[particle_counts < max_particle_select]
        X = X[particle_counts < max_particle_select]

    print("X: ", X.shape, X.dtype)
    print("Y: ", Y.shape, Y.dtype)

    return X, Y, event_ids

In [ ]:
y_dim = 1
x_dim = 3



# Dataset Parameters. Change cache_dir to your local path where the data will be stored.
cache_dir = "Dataset/CMS-Open-Data"
momentum_scale = 1000
n = 100000
pad = 150
pt_lower, pt_upper = 500, 1000
eta = 2.4
quality = 2

# #############################
# ########## DATASET ##########
# #############################

X, Y, ids = load_data(cache_dir, pt_lower, pt_upper, eta, quality, pad, momentum_scale = momentum_scale, n = n, max_particle_select = None, amount = 1)
X_test, Y_test, ids_test = load_data(cache_dir, pt_lower, pt_upper, eta, quality, pad, momentum_scale = momentum_scale, n = 50)


In [ ]:
sorted = np.sort(ids)
sorted_indices = ids.argsort()
print(ids.shape)
print(sorted)


counter = 0
pairs = []
N = len(sorted)
for (i,id) in enumerate(sorted):
    for (j, id2) in enumerate(sorted[(i+1):]):

        if id == id2:
            counter += 1
            pairs.append((i, i+1+j))
            break

        if id2 > id:
            break

print(counter / N )

In [6]:
sorted_gen_pts = Y[sorted_indices]
sorted_sim_pts = X[sorted_indices,0]

leading_gen_pts = []
subleading_gen_pts = []
leading_sim_pts = []
subleading_sim_pts = []

for pair in pairs:

    jet1 = sorted_gen_pts[pair[0]]
    jet2 = sorted_gen_pts[pair[1]]

    sim_jet1 = sorted_sim_pts[pair[0]]
    sim_jet2 = sorted_sim_pts[pair[1]]

    leading_gen_pts.append(jet2)
    subleading_gen_pts.append(jet1)
    leading_sim_pts.append(sim_jet2)
    subleading_sim_pts.append(sim_jet1)

leading_gen_pts = np.array(leading_gen_pts)[:,0]
subleading_gen_pts = np.array(subleading_gen_pts)[:,0]

leading_sim_pts = np.array(leading_sim_pts)
subleading_sim_pts = np.array(subleading_sim_pts)

### Experimental Data (Nature)

In [7]:
x_data = leading_gen_pts + subleading_gen_pts
x_data = x_data.reshape(-1,1).astype(np.float64)

In [8]:
leading_sim_pts_hold = leading_gen_pts + theta*(leading_sim_pts - leading_gen_pts)
subleading_sim_pts_hold = subleading_gen_pts + theta*(subleading_sim_pts-subleading_gen_pts)
y_data = np.c_[leading_sim_pts_hold + subleading_sim_pts_hold, leading_sim_pts_hold - subleading_sim_pts_hold]

### Monte Carlo Data (Simulation)

In [9]:
def weight_func(x):
    return x[0]*np.exp(x[0])

weights = np.apply_along_axis(weight_func, axis=1, arr=x_data)

# Normalize weights
weights /= np.sum(weights)

# Perform weighted sampling
N_mc = len(x_data)  # Number of samples to draw
idx = np.random.choice(np.arange(len(x_data)), size=N_mc, replace=True, p=weights)

In [10]:
x_mc = leading_gen_pts[idx] + subleading_gen_pts[idx]
x_mc = x_mc.reshape(-1,1).astype(np.float64)
y_mc = np.c_[leading_sim_pts[idx] + subleading_sim_pts[idx], leading_sim_pts[idx] - subleading_sim_pts[idx]].astype(np.float64)

### Plot both experimental and MC distributions

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 6))

sns.kdeplot(
    x=x_data[:, 0], ax=ax[0],
    color="black", linestyle="-", linewidth=2, bw_adjust=2,
    label=f"Experiment"
)

sns.kdeplot(
    x=x_mc[:, 0], ax=ax[0],
    color="tab:blue", linestyle=":", linewidth=2, bw_adjust=2, 
    label=f"MC Simulation"
)

sns.kdeplot(
    x=y_data[:, 0], ax=ax[1],
    color="black", linestyle="-", linewidth=2, bw_adjust=2,
    label=f"Experiment"
)

sns.kdeplot(
    x=y_mc[:, 0], ax=ax[1],
    color="tab:blue", linestyle=":", linewidth=2, bw_adjust=2, 
    label=f"MC Simulation"
)

sns.kdeplot(
    x=y_data[:, 1], ax=ax[2],
    color="black", linestyle="-", linewidth=2, bw_adjust=2,
    label=f"Experiment"
)

sns.kdeplot(
    x=y_mc[:, 1], ax=ax[2],
    color="tab:blue", linestyle=":", linewidth=2, bw_adjust=2, 
    label=f"MC Simulation"
)


ax[2].legend(loc="upper right", fontsize=15)
ax[0].set_xlim(0.8, 2.0)
ax[1].set_xlim(0.8, 2.0)
ax[2].set_xlim(-0.5, 0.5)
ax[0].set_xlabel(r"$X$", fontsize=20)
ax[1].set_xlabel(r"$Y_1$", fontsize=20)
ax[2].set_xlabel(r"$Y_2$", fontsize=20)
ax[0].set_ylabel("Probability Density", fontsize=20)
ax[0].tick_params(axis="both", labelsize=14)
fig.tight_layout()

Our goal is to reweight the MC density of $X$ (in <span style="color:blue">blue</span>) to match the unknown density of data $X$ (in <span style="color:black">black</span>).

## Train W model

We train a neural network (NN) model to learn the W function, which represents the ratio of the response kernel parametrized by $\theta$ to the Monte Carlo (MC) kernel, i.e. $w(y,x,\theta)=p(y|x,\theta)/q(y|x)$.

In [31]:
# hyperparameters for training the neural network
config = {
    'batch_size': 10000,
    'lr': 0.001,
    'patience': 30,
    'activation': nn.ReLU()
}

### Systematic Data (varying theta, used for training W function)

In [11]:
# specify the range of training theta
theta_min = 0.5
theta_max = 2.0
N_sys = x_mc.shape[0]

theta0_sim = np.random.uniform(theta_min, theta_max, N_mc).reshape(-1,1)
theta1_sim = np.random.uniform(theta_min, theta_max, N_sys).reshape(-1,1)

In [12]:
leading_sim_pts_hold = leading_gen_pts + theta1_sim[:,0]*(leading_sim_pts - leading_gen_pts)
subleading_sim_pts_hold = subleading_gen_pts + theta1_sim[:,0]*(subleading_sim_pts-subleading_gen_pts)
y_sys = np.c_[leading_sim_pts_hold + subleading_sim_pts_hold, leading_sim_pts_hold - subleading_sim_pts_hold]

### Train a single W function

In [14]:
w_ds = pof.w_dataset(x_mc, y_mc, theta0_sim, x_data, y_sys, theta1_sim)

w_ds_train, w_ds_test = random_split(w_ds, [len(w_ds)//2, len(w_ds)-len(w_ds)//2])
w_dataloader_train = DataLoader(w_ds_train, batch_size=10000, shuffle=True, num_workers=0)
w_dataloader_test = DataLoader(w_ds_test, batch_size=10000, shuffle=False, num_workers=0)

In [ ]:
# Train W model

wRT_model_network = pof.wRT_network(sigmoid=True, n_inputs=4, activation=config['activation']).double().to(dvc)
optimizerRT = optim.Adam(wRT_model_network.parameters(), lr=config['lr'])
loss_fn_RT = nn.BCELoss()
wRT_tr = pof.w_trainer(w_dataloader_train, w_dataloader_test, wRT_model_network, loss_fn_RT, optimizerRT, patience=config['patience'])

wT_model_network = pof.wT_network(sigmoid=True, n_inputs=2, activation=config['activation']).double().to(dvc)
optimizerT = optim.Adam(wT_model_network.parameters(), lr=config['lr'])
loss_fn_T = nn.BCELoss()
wT_tr = pof.w_trainer(w_dataloader_train, w_dataloader_test, wT_model_network, loss_fn_T, optimizerT)

wRT_tr.fit()
wT_tr.fit()

In [ ]:
# optionally, save the models for later access
wRT_checkpoint = {
    "model_state_dict": wRT_model_network.state_dict(),
    "config": config,
    "num_sys": N_sys,
    "num_mc": N_mc,
    "theta_min": theta_min,
    "theta_max": theta_max
}
wT_checkpoint = {
    "model_state_dict": wT_model_network.state_dict(),
    "config": config,
    "num_sys": N_sys,
    "num_mc": N_mc,
    "theta_min": theta_min,
    "theta_max": theta_max
}

torch.save(wRT_checkpoint, "models/OpenData/wRT_network_opendata.pth")
torch.save(wT_checkpoint, "models/OpenData/wT_network_opendata.pth")

### Train an ensemble of W functions

In [ ]:
for i in range(1,11):
    # MC data
    N_mc = len(x_data)  # Number of samples to draw
    idx = np.random.choice(np.arange(len(x_data)), size=N_mc, replace=True, p=weights)
    x_mc = np.c_[leading_gen_pts[idx] + subleading_gen_pts[idx], leading_gen_pts[idx] - subleading_gen_pts[idx]].astype(np.float64)
    y_mc = np.c_[leading_sim_pts[idx] + subleading_sim_pts[idx], leading_sim_pts[idx] - subleading_sim_pts[idx]].astype(np.float64)

    # Systematic data
    theta_min = 0.5
    theta_max = 1.5
    N_sys = x_mc.shape[0]
    
    theta0_sim = np.random.uniform(theta_min, theta_max, N_mc).reshape(-1,1)
    theta1_sim = np.random.uniform(theta_min, theta_max, N_sys).reshape(-1,1)
    
    leading_sim_pts_hold = leading_gen_pts + theta1_sim[:,0]*(leading_sim_pts - leading_gen_pts)
    subleading_sim_pts_hold = subleading_gen_pts + theta1_sim[:,0]*(subleading_sim_pts-subleading_gen_pts)
    Y_sys = np.c_[leading_sim_pts_hold + subleading_sim_pts_hold, leading_sim_pts_hold - subleading_sim_pts_hold]
    
    w_ds = pof.w_dataset(x_mc[:,0:1], y_mc, theta0_sim, x_data[:,0:1], Y_sys, theta1_sim)
    
    w_ds_train, w_ds_test = random_split(w_ds, [len(w_ds)//2, len(w_ds)-len(w_ds)//2])
    w_dataloader_train = DataLoader(w_ds_train, batch_size=10000, shuffle=True, num_workers=0)
    w_dataloader_test = DataLoader(w_ds_test, batch_size=10000, shuffle=False, num_workers=0)
    
    # Train W model
    
    wRT_model_network = pof.wRT_network(sigmoid=True, n_inputs=4, activation=config['activation']).double().to(dvc)
    optimizerRT = optim.Adam(wRT_model_network.parameters(), lr=config['lr'])
    loss_fn_RT = nn.BCELoss()
    wRT_tr = pof.w_trainer(w_dataloader_train, w_dataloader_test, wRT_model_network, loss_fn_RT, optimizerRT, patience=config['patience'])
    
    wT_model_network = pof.wT_network(sigmoid=True, n_inputs=2, activation=config['activation']).double().to(dvc)
    optimizerT = optim.Adam(wT_model_network.parameters(), lr=config['lr'])
    loss_fn_T = nn.BCELoss()
    wT_tr = pof.w_trainer(w_dataloader_train, w_dataloader_test, wT_model_network, loss_fn_T, optimizerT)
    
    wRT_tr.fit()
    wT_tr.fit()
    
    torch.save(wRT_model_network.state_dict(), f"models/OpenData/Ensemble/wRT_network_opendata({i})")
    torch.save(wT_model_network.state_dict(), f"models/OpenData/Ensemble/wT_network_opendata({i})")

## Load W model

In [ ]:
# load the models if saved previously
wRT_model_network = pof.wRT_network(sigmoid=True, n_inputs=4).double().to(dvc)
wT_model_network = pof.wT_network(sigmoid=True, n_inputs=2).double().to(dvc)

wRT_model_network.load_state_dict(torch.load("models/OpenData/wRT_network_opendata.pth")["model_state_dict"])
wT_model_network.load_state_dict(torch.load("models/OpenData/wT_network_opendata.pth")["model_state_dict"])

In [ ]:
# load the ensemble of models
wRT_list = glob.glob("models/OpenData/Ensemble/wRT_network_opendata(*)")
wT_list = glob.glob("models/OpenData/Ensemble/wT_network_opendata(*)")

wRT_ensemble = []
wT_ensemble = []
for i in range(len(wRT_list)):
    wRT_ensemble.append(pof.wRT_network(sigmoid=True, activation=config['activation'], n_inputs=4).double().to(dvc))
    wT_ensemble.append(pof.wT_network(sigmoid=True, activation=config['activation'], n_inputs=2).double().to(dvc))
    wRT_ensemble[i].load_state_dict(torch.load(wRT_list[i]))
    wT_ensemble[i].load_state_dict(torch.load(wT_list[i]))

In [57]:
# wrap the w function on the MC dataset (so that it becomes only a function of theta)
ds = pof.test_dataset(x_mc, y_mc)
ds_dataloader = DataLoader(ds, batch_size=100000, shuffle=False)
#w_theta_nn = pof.make_w_theta(ds_dataloader, wRT_model_network, wT_model_network)
w_theta_nn_ensemble = pof.make_w_theta_ensemble(ds_dataloader, wRT_ensemble, wT_ensemble, func='median')

## Profile OmniFold Algorithm

Now, let's run the algorithm!

In [ ]:
# use the NN fitted w function
#w_theta = w_theta_nn
w_theta = w_theta_nn_ensemble

In [ ]:
# Run POF with a single initial theta value
theta0 = 1.0
pof_out = pof.profile_omnifold(y_data, x_mc, y_mc, iterations=10, w_theta=w_theta, theta_bar=1.0, theta0=theta0, theta_range=[1.0,2.0], num_grid_points=30,
                                               no_penalty=True, epochs=20, patience=3, verbose=0)
nu_pof = pof_out['weights']

In [ ]:
# Generate Ensemble of POF solutions with different initial theta0
theta0_list = np.arange(1.0, 1.9, 0.1)
print('theta0_list:', np.round(theta0_list,1))
pof_list = []
for theta0 in theta0_list:
    res = pof.profile_omnifold(y_data, x_mc, y_mc, iterations=10, w_theta=w_theta, theta_bar=1.0, theta0=theta0, theta_range=[0.5, 1.5], 
                                                       num_grid_points=50, no_penalty=True, epochs=20, patience=3,
                                                       return_Q=True, return_acc=True, return_loss=True, verbose=0)
    pof_list.append(res)

In [ ]:
# updated theta in each iteration
temp = np.arange(1,11,dtype=int)
columns = ['iteration'] + ['theta0: '] * len(pof_list)
for i in range(0, len(pof_list)):
    temp = np.vstack((temp, pof_list[i]['weights'][:,3,0]))
    columns[i+1] += str(np.round(pof_list[i]['theta0'], 1))
df_theta = pd.DataFrame(temp.T, columns=columns)

# validation accuracy of step 1 neural network
temp = np.arange(1,11,dtype=int)
columns = ['iteration'] + ['theta0: '] * len(pof_list)
for i in range(0, len(pof_list)):
    temp = np.vstack((temp, pof_list[i]['step1_val_acc'].ffill(axis=1).iloc[:,-1]))
    columns[i+1] += str(np.round(pof_list[i]['theta0'], 1))
df_acc = pd.DataFrame(temp.T, columns=columns)

In [ ]:
# theta and accuracy evolution

# Normalize values to colormap range
norm = mcolors.Normalize(vmin=min(theta0_list), vmax=max(theta0_list))
cmap = cm.viridis_r

# Plot each theta curve
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(10, 8), sharex=True)
for i in range(1,len(df_theta.columns)):
    ax1.plot([0]+df_theta['iteration'].tolist(), [pof_list[i-1]['theta0']] + df_theta.iloc[:,i].tolist(), 
             color=cmap(norm(theta0_list[i-1])))


ax1.axhline(y=theta, color='red', linestyle='--', linewidth=1.2, label=rf'Truth: $\theta={theta}$')
ax1.legend(loc='upper right')
ax1.set_ylabel(r"$\hat{\theta}$", size=16)
ax1.set_title(r"$\hat{\theta}$ update in EM iteration", size=16)
ax1.tick_params(axis='both', labelsize=14) 
ax1.grid(True)

# Plot each goodness_of_fit curve
for i in range(1,len(df_acc.columns)):
    ax2.plot(df_theta['iteration'], 1 - np.abs(df_acc.iloc[:,i]-0.5) * 2, color=cmap(norm(theta0_list[i-1])))


ax2.set_xlabel("Iteration", size=16)
ax2.set_ylabel("Goodness of fit", size=14)
ax2.set_title("Goodness of fit in EM iteration", size=16)
ax2.tick_params(axis='both', labelsize=14) 
ax2.grid(True)


sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # dummy for colorbar
cbar = fig.colorbar(sm, ax=[ax1, ax2], orientation='vertical', fraction=0.025, pad=0.02)
cbar.set_label(r'$\theta_0$', size=16)

In [9]:
# indicate which iteration to plot
itr = -1

# find the best weights (fit) by the classifier accurarcy (closer to 0.5 indicates a better fit)
nu_pof, best_run = pof.best_weights(pof_list, itr)
pof_out = pof_list[best_run]

In [ ]:
print("chosen run:", best_run)
print("theta0:", pof_out['theta0'])
print("fitted theta:", nu_pof[itr,3,0])
print("true theta:", theta)

In [ ]:
# As a comparison, we can also run the vanilla omnifold algorithm
of_out = pof.omnifold(y_data, x_mc, y_mc, iterations=10, verbose=0, epochs=20)
nu_of = of_out['weights']

## Plot the results

In [ ]:
# indicate which iteration to plot
itr = -1

fig, ax = plt.subplots(1, 1, sharey=True, sharex=True, figsize=(10, 7))

sns.kdeplot(x=x_data[:,0], ax=ax, color="black",label=rf'Experiment', linestyle="-", bw_adjust=2, lw=2)
sns.kdeplot(x=x_mc[:,0], ax=ax, color="tab:blue",label=rf'MC Simulation', linestyle=":", bw_adjust=2, lw=2)
sns.kdeplot(x=x_mc[:,0], ax=ax, color="tab:green", weights=nu_of[itr,1,:], label=rf"OmniFold ($\theta=1$)", linestyle="--", bw_adjust=2, lw=2)
sns.kdeplot(x=x_mc[:,0], ax=ax, color="tab:orange", weights=nu_pof[itr,1,:],label=rf"Profile OmniFold ($\hat{{\theta}}={nu_pof[itr,3,0]:.2f}$)", linestyle="-.", bw_adjust=2, lw=2)



ax.legend(loc="best", fontsize=15, frameon=False)
ax.set_xlim(0.8, 2.0)
ax.set_xlabel(r"$X$ [TeV]", fontsize=20)
ax.set_ylabel("Probability Density", fontsize=20)
ax.tick_params(axis="both", labelsize=14)
fig.tight_layout()
plt.legend(loc='best',fontsize=10)

In [ ]:
# Y1
cpwr = utils.comparison_plots_with_ratio(0.8, 2.0, 50, xlabel=r"$Y_1$ [TeV]", density=True, legend_corner="best", header="")
cpwr.add_data(y_data[:,0], label=rf"Experiment ($\theta={theta}$)", target=True, color='black', histtype="step", ls="-", lw=2)
cpwr.add_data(y_mc[:,0], label=rf"MC Simulation ($\theta=1$)", histtype="step", ls=":", lw=2, color='tab:blue')
cpwr.add_data(y_mc[:,0], weights=nu_of[itr,0,:], label=r"OmniFold ($\theta=1$)", histtype="step", ls="--", lw=2, color='tab:green')
cpwr.add_data(y_mc[:,0], weights=nu_pof[itr,0,:]*nu_pof[itr,2,:], label=rf'ProfileOmniFold ($\hat{{\theta}}={nu_pof[itr,3,0]:.2f}$)', histtype="step", ls="-.", lw=2, color='tab:orange')
cpwr.show()

# Y2
cpwr = utils.comparison_plots_with_ratio(-0.2, 0.2, 50, xlabel=r"$Y_2$ [TeV]", density=True, legend_corner="best", header="")
cpwr.add_data(y_data[:,1], label=rf"Experiment ($\theta={theta}$)", target=True, color='black', histtype="step", ls="-", lw=2)
cpwr.add_data(y_mc[:,1], label=rf"MC Simulation ($\theta=1$)", histtype="step", ls=":", lw=2, color='tab:blue')
cpwr.add_data(y_mc[:,1], weights=nu_of[itr,0,:], label=r"OmniFold ($\theta=1$)", histtype="step", ls="--", lw=2, color='tab:green')
cpwr.add_data(y_mc[:,1], weights=nu_pof[itr,0,:]*nu_pof[itr,2,:], label=rf'ProfileOmniFold ($\hat{{\theta}}={nu_pof[itr,3,0]:.2f}$)', histtype="step", ls="-.", lw=2, color='tab:orange')
cpwr.show()

# X
cpwr = utils.comparison_plots_with_ratio(0.8, 2.0, 50, xlabel=r"$X$ [TeV]", density=True, legend_corner="best", header="")
cpwr.add_data(x_data, label="Experiment", target=True, color='black', histtype="step", ls="-", lw=2)
cpwr.add_data(x_mc, label="MC Simulation", histtype="step", ls=":", lw=2, color='tab:blue')
cpwr.add_data(x_mc, weights=nu_of[itr,1,:], label=r"OmniFold ($\theta=1$)", histtype="step", ls="--", lw=2, color='tab:green')
cpwr.add_data(x_mc, weights=nu_pof[itr,1,:], label=rf'ProfileOmniFold ($\hat{{\theta}}={nu_pof[itr,3,0]:.2f}$)', histtype="step", ls="-.", lw=2, color='tab:orange')
cpwr.show()

print('fitted theta:', nu_pof[itr,3,0])